In [97]:
import numpy as np
import plotly as plt
from scipy.optimize import minimize
from tqdm import tqdm

In [98]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

In [99]:
eps = 0.0012   # Per share fee parameter
delta = 0.005   # tick size
gamma = 5 # inventory penelisation 
m = 6  # the number of states of the spread
n = 100 # grid of inventory
e = 100 # max volumne for market orders
T = 300 # timespan
dt = T / 100 # time difference for Euler scheme
l_max = 100 # maximum volume of an ordinary order
initial_x = [50, 0] # initial guess for optimisation inside the differential equation

In [100]:
intensity_matrix_a = np.array([[0.0539, 0.1485], 
                              [0.0465, 0.0979],
                              [0.0401, 0.0846],
                              [0.0360, 0.0856],
                              [0.0435, 0.1009],
                              [0.0554, 0.1202]]) # Estimated intensities for given quotes and spread for ask

intensity_matrix_b = np.array([[0.0718, 0.1763], 
                              [0.0520, 0.1144],
                              [0.0419, 0.0915],
                              [0.0409, 0.0896],
                              [0.0452, 0.0930],
                              [0.0614, 0.1255]])    # Estimated intensities for given quotes and spread for bid

intensity_matrix_sym = (intensity_matrix_a + intensity_matrix_b) / 2    # Making bid and ask sides symmetric

In [101]:
prob_matrix = np.array([[0, 0.41, 0.22, 0.16, 0.142, 0.065], 
                        [0.201, 0, 0.435, 0.192, 0.103, 0.067],
                        [0.113, 0.221, 0, 0.4582, 0.147, 0.059],
                        [0.07, 0.085, 0.275, 0, 0.465, 0.102],
                        [0.068, 0.049, 0.073, 0.363, 0, 0.446],
                        [0.077, 0.057, 0.059, 0.112, 0.692, 0]])   # probability transition matrix
R = prob_matrix
R[np.arange(0, R.shape[0]), np.arange(0, R.shape[0])] = -np.sum(R, axis = 1)  # intensity matrix of Markov chain that determines the spread process

In [102]:
i = 1 # spread position
y = np.linspace(-1000, 1000, n) # mesh grid in inventory

In [103]:
def g(y):
    return y ** 2

In [104]:
def phi_objective(x, i, delta, phi_t, y, y_current, intensity_matrix, bid_or_ask):
    
    l = x[0]
    q = 1 if 1 == 1 else 0

    lambda_phi = intensity_matrix[i, q]

    if bid_or_ask == "ask":
        y_shift = y_current - l
    else:
        y_shift = y_current + l

    #y_broadcasted = y.reshape(1, -1)
    #shifted_y_broadcasted = y_shift.reshape(-1, 1)
    #y_difference = np.abs(y_broadcasted - shifted_y_broadcasted)
    #y_index = np.argmin(y_difference, axis = 1)
    y_index_l = np.argmin(np.abs(y - y_shift))
    y_index = np.argmin(np.abs(y - y_current))
    phi_y_l = phi_t[y_index_l]
    phi_t = phi_t[y_index]

    if bid_or_ask == "ask":
        objective = lambda_phi * (phi_y_l - phi_t + (i *  delta / 2 - delta * q) * l)
    else:
        objective = lambda_phi * (phi_y_l - phi_t + (i *  delta / 2 - delta * q) * l)
    return -objective

In [105]:
def optimise_phi(y, y_current, initial_x, phi_t, delta, i, intensity_matrix, constraints, bid_or_ask):
    

    solution = minimize(phi_objective, initial_x, bounds = [(0, l_max), (-1, 1)], constraints = constraints, tol = 1e-16,
                                args = (i, delta, phi_t, y, y_current, intensity_matrix, bid_or_ask))
                            
    f = -solution.fun
    solution = solution.x

    return (solution, f)

In [106]:
phi_terminal = -np.abs(y) * i * delta / 2 - eps
constraints = ({'type': 'eq', 'fun': lambda x: 1 - x[1]**2})

In [107]:
phi = [phi_terminal]
q_b = [[0 for 0 in range(phi_terminal)]]
l_b = [[0 for 0 in range(phi_terminal)]]
q_a = [[0 for 0 in range(phi_terminal)]]
l_a = [[0 for 0 in range(phi_terminal)]]
phi_temp = phi_terminal

for t in tqdm(range(1, int(T / dt))):

    phi_ask_result = []
    phi_bid_result = []
    q_b_temp = []
    l_b_temp = []
    q_a_temp = []
    l_a_temp = []

    for y_current in np.nditer(y):
        phi_opt_ask = optimise_phi(y_current = y_current, y = y, initial_x = initial_x, phi_t = phi_temp, 
                                    delta = delta, i = i, intensity_matrix = intensity_matrix_sym, constraints = constraints, bid_or_ask = "ask")
        phi_opt_bid = optimise_phi(y_current = y_current, y = y, initial_x = initial_x, phi_t = phi_temp, 
                                    delta = delta, i = i, intensity_matrix = intensity_matrix_sym, constraints = constraints, bid_or_ask = "bid")
        phi_ask_result.append(phi_opt_ask[1])
        l_b_temp.append(phi_opt_ask[0][0])
        q_b_temp.append(phi_opt_ask[0][1])
        phi_bid_result.append(phi_opt_bid[1])
        l_a_temp.append(phi_opt_bid[0][0])
        q_a_temp.append(phi_opt_bid[0][1])

    phi_temp = -dt * (np.array(phi_ask_result) + np.array(phi_bid_result) - gamma * g(y))
    phi.append(phi_temp)
    q_b.append(q_b_temp)
    l_b.append(l_b_temp)
    q_a.append(q_a_temp)
    l_a.append(l_b_temp)


phi.reverse()

100%|██████████| 99/99 [00:56<00:00,  1.75it/s]


In [144]:
optimise_phi(y, y_current, initial_x, phi_terminal, delta, i, intensity_matrix_sym, constraints, "bid")[0][1]

1.0